In [1]:
import math
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from haversine import haversine, Unit

tqdm.pandas(desc='Progress')

In [2]:
def correct_rounding(number):
    if number > 0:
        return int(number + 0.5)
    else:
        return int(number + -0.5)
    
def create_coordinate_list (lat_matrix, lon_matrix):
    N, M = lat_matrix.shape
    coordinate_list = []
    for i in range(N):
        for j in range(M):
            coordinate_list.append([lat_matrix[i, j], lon_matrix[i, j]])
    return coordinate_list

def euclidean_distance(point1: list, point2: list):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))

def deg2rad(deg):
    return deg * (math.pi / 180.0)

def get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    r = 6371.0
    d_lat = deg2rad(lat2 - lat1)
    d_lon = deg2rad(lon2 - lon1)
    a = (math.sin(d_lat / 2.0) * math.sin(d_lat / 2.0) + math.cos(deg2rad(lat1))
         * math.cos(deg2rad(lat2)) * math.sin(d_lon / 2.0) * math.sin(d_lon / 2.0))
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))
    d = r * c
    return d

def find_closest_coordinates(point: list, coordinates_list: list):
    min_dist = float('inf')
    closest_coord = None
    for coords in coordinates_list:
        eucl_distances = get_distance_from_lat_lon_in_km(*point, *coords)
        if eucl_distances <= min_dist:
            min_dist = eucl_distances
            closest_coord = coords
    return closest_coord   


def find_index (lat_matrix, lon_matrix, target_coord):
    target_lat = target_coord[0]
    target_lon = target_coord[1]
    N, M = lat_matrix.shape
    result_index = None
    
    for i in range(N):
        for j in range(M):
            lat = lat_matrix[i, j]
            lon = lon_matrix[i, j]
            if lat == target_lat and lon == target_lon:
                result_index = i, j
    return result_index

def get_correct_lon(lon: int):
    if lon > 180:
        lon = lon - 360
    return lon

### Подгрузка данных

In [3]:
sheets = pd.ExcelFile('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx').sheet_names
lon = pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name='lon')
lon = lon.applymap(get_correct_lon)
lon = np.array(lon)
lat = np.array(pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name='lat'))
week_data_dict = {}
for sheet in tqdm(sheets[2:]):
    data = pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name=sheet)
    data = data.applymap(correct_rounding)
    # coordinates_integr_velocity = np.array([lat, lon, data])
    week_data_dict[sheet] = data
    
lat_lon_coordinates = create_coordinate_list(lat, lon)

/tmp/ipykernel_6940/476155299.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lon = lon.applymap(get_correct_lon)
  0%|          | 0/14 [00:00<?, ?it/s]/tmp/ipykernel_6940/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
  7%|▋         | 1/14 [00:00<00:05,  2.19it/s]/tmp/ipykernel_6940/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 14%|█▍        | 2/14 [00:00<00:04,  2.54it/s]/tmp/ipykernel_6940/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 21%|██▏       | 3/14 [00:01<00:04,  2.42it/s]/tmp/ipykernel_6940/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 29%|██▊       | 4/14 [00:01<00:

In [4]:
reference_graph_points = pd.read_excel('/home/kmulygin/khakaton/data/ГрафДанные.xlsx', sheet_name = 'points')
reference_graph_edges = pd.read_excel('/home/kmulygin/khakaton/data/ГрафДанные.xlsx', sheet_name = 'edges')

### Нахождение необходимых координат

In [5]:
def get_graph_index(initial_coordinates: list):
    closest_coordinate = find_closest_coordinates(initial_coordinates, lat_lon_coordinates)
    return find_index(lat, lon, closest_coordinate)

In [6]:
dict_pointid_lat_lon = {}
for _, row in reference_graph_points.iterrows():
    dict_pointid_lat_lon[row['point_id']] = [[row['latitude'], row['longitude']], row['point_name']]

In [7]:
start_data = reference_graph_edges[['start_point_id', 'end_point_id', 'length']]
start_coordinates = []
end_coordinates = []
name_start = []
name_end = []
for _, row in start_data.iterrows():
    start_coordinates.append(dict_pointid_lat_lon[row['start_point_id']][0])
    name_start.append(dict_pointid_lat_lon[row['start_point_id']][1])
    end_coordinates.append(dict_pointid_lat_lon[row['end_point_id']][0])
    name_end.append(dict_pointid_lat_lon[row['end_point_id']][1])

start_data['start_coordinate'] = start_coordinates
start_data['end_coordinate'] = end_coordinates
start_data['name_start'] = name_start
start_data['name_end'] = name_end
start_data['graph_start_index'] = start_data['start_coordinate'].progress_apply(lambda x: get_graph_index(x))
start_data['graph_end_index'] = start_data['end_coordinate'].progress_apply(lambda x: get_graph_index(x))

/tmp/ipykernel_6940/977449582.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_data['start_coordinate'] = start_coordinates
/tmp/ipykernel_6940/977449582.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_data['end_coordinate'] = end_coordinates
/tmp/ipykernel_6940/977449582.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

### Построение графа и визуализация

In [8]:
# для ледоколов необходимо прописывать отдельное ограничение
def get_ice_graph (speed_matrix: np.array, ship_type: str, ship_name: str, max_personal_speed: float) -> nx.DiGraph:    
    N, M= speed_matrix.shape
   
    G = nx.DiGraph()
    # Добавляем ребра и веса
    if ship_type == 'Нет':
        for i in range(N):
            for j in range(M):
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist
                        # weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 4' or ship_type == 'Arc 5' or ship_type == 'Arc 6':
        for i in range(N):
            for j in range(M):
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 7':
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * speed_matrix[i+1, j], max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * speed_matrix[i+1, j], max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 9' and (ship_name == '50 лет Победы' or ship_name == 'Ямал'):
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            # weight = dist / avg_speed
                            weight = dist
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
    elif  ship_type == 'Arc 9' and (ship_name == 'Вайгач' or ship_name == 'Таймыр'):
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        
                        
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        # weight = dist / avg_speed
                        weight = dist
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                
    return G


In [9]:
def graph_visualize(speed_matrix: np.array, graph: nx.classes.digraph.DiGraph):
    
    N, M = speed_matrix.shape
    pos = {(i, j): (j, -i) for i in range(N) for j in range(M)}  # расположение узлов для визуализации
    labels = {node: f"{node}\n{speed_matrix[node[0], node[1]]} км/ч" for node in graph.nodes()}
    edge_labels = {(u, v): f"{d['weight']:.2f} ч" for u, v, d in graph.edges(data=True)}

    print('Отрисовка графика начата')
    plt.figure(figsize=(6, 6))
    nx.draw(graph, pos, with_labels=True, labels=labels, node_size=2000, node_color="lightblue", font_size=10, font_color="black", font_weight="bold")
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_color='red')
    plt.title("Граф возможных перемещений с весами ребер (время в часах)")
    plt.show()

### Аглоритм Дейкстры

In [10]:
def get_path(graph, index_start, index_end):
    try:
        _, path = nx.single_source_dijkstra(graph, index_start, index_end, weight='weight')
        return path
    except:
        return 'Нужен ледокол'

def get_cost(graph, index_start, index_end):
    try:
        cost, _ = nx.single_source_dijkstra(graph, index_start, index_end, weight='weight')
        return cost
    except:
        return 'Нужен ледокол'

### Тестирование

In [11]:
ship_types = ['Нет', 'Arc 4', 'Arc 7', 'Arc 9']
ship_names = ['Имя_судна_1', 'Имя_судна_2', 'Имя_судня_3', 'Ямал']
ship_speeds = [14, 19, 19, 21, 18.5]

In [12]:
result_model_output = {}
for week in tqdm(week_data_dict):
    result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        result_model_output[week][sheep_types_name] = {}
        result_model_output[week][sheep_types_name]['path'] = {}
        result_model_output[week][sheep_types_name]['costs'] = {}
        Graph = get_ice_graph(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])
        # Graph_data = nx.node_link_data(Graph)
        # with open(f"graphs_week_sheep_types/{week}_{ship_types[i]}_graph.json", "w") as f:
        #     json.dump(Graph_data, f)   
        for _, row in start_data.iterrows():
            result_model_output[week][sheep_types_name]['path'][f"{row['name_start']} + {row['name_end']}"] = get_path(Graph, row['graph_start_index'], row['graph_end_index'])
            result_model_output[week][sheep_types_name]['costs'][f"{row['name_start']} + {row['name_end']}"] = get_cost(Graph, row['graph_start_index'], row['graph_end_index'])

100%|██████████| 14/14 [01:29<00:00,  6.40s/it]


### Тестирование подходы для создания ледового графа

In [13]:
Graph_ice3 = get_ice_graph(speed_matrix=np.array(week_data_dict['03-Mar-2020']), ship_type='Нет', ship_name='Простое судно', max_personal_speed=14)
Graph_arc_9_yamal = get_ice_graph(speed_matrix=np.array(week_data_dict['03-Mar-2020']), ship_type='Arc 9', ship_name='Ямал', max_personal_speed=21)

In [14]:
list_9_start = list(Graph_arc_9_yamal.nodes())
print(len(list_9_start))
list_ice_3 = list(Graph_ice3.nodes())
print(len(list_ice_3))

8904
2688


In [15]:
len(set(list_9_start) & set(list_ice_3))

2688

In [16]:
for node in list(Graph_arc_9_yamal.nodes()):
    if node not in Graph_ice3.nodes():
        continue
    elif Graph_arc_9_yamal.degree(node) >  Graph_ice3.degree(node):
        continue
    else:
        Graph_arc_9_yamal.remove_node(node)
print(len(list(Graph_arc_9_yamal.nodes())))

6605


### Решение проблем с координатами некоторых портов для ледокола Ямал

In [17]:
bad_ports = {}
for week in list(week_data_dict.keys()):
    bad_ports[week] = []
    for key, values in result_model_output[week]['Arc 9_Ямал']['costs'].items():
        if values == 'Нужен ледокол':
            bad_ports[week].append(key)
    bad_ports[week] = list(set(bad_ports[week]))

In [18]:
list_bad_routes = []
compare = bad_ports['03-Mar-2020']
for week in bad_ports:
    for route in bad_ports[week]:
        list_bad_routes.append(route)
        
list_bad_ports = []
for ports in list_bad_routes:
    ports = ports.split(' + ')
    for port in ports:
        list_bad_ports.append(port)
        
list_bad_ports = list(set(list_bad_ports))

dict_bad_ports_coordinates = {}
for port in list_bad_ports:
    for _, row in start_data.iterrows():
        if row['name_start'] == port:
            dict_bad_ports_coordinates[port] = row['graph_start_index']
        elif row['name_end'] == port:
            dict_bad_ports_coordinates[port] = row['graph_end_index']

In [19]:
all_date_coord_list = {}

for week in week_data_dict:
    N, M = week_data_dict[week].shape
    matrix = np.array(week_data_dict[week])
    coord_list = []
    for i in range(N):
        for j in range(M):
            if matrix[i, j] >= 10:
                coord_list.append((i, j))
    all_date_coord_list[week] = coord_list

In [20]:
target_port = 'Индига'
target_coordinate = (27, 48)
shortest_point_for_bad_ports  = {}

for week in tqdm(list(week_data_dict.keys())):
    coord_list = all_date_coord_list[week]
    shortest_point_for_bad_ports[week] = {}
    Graph = get_ice_graph(speed_matrix=np.array(week_data_dict[week]), max_personal_speed=21, ship_name='Ямал', ship_type='Arc 9')
    # with open('khakaton/shortest_point_for_bad_ports.pkl', 'wb') as f:
    #     pickle.dump(shortest_point_for_bad_ports, f)
    for port in dict_bad_ports_coordinates:
        min_distance = float('inf')
        path = get_path(graph=Graph, index_start=target_coordinate, index_end=dict_bad_ports_coordinates[port])
        if path == 'Нужен ледокол':
            for coord in coord_list:
                distance = haversine((lat[coord[0], coord[1]], lon[coord[0], coord[1]]), (lat[dict_bad_ports_coordinates[port][0], dict_bad_ports_coordinates[port][1]], lon[dict_bad_ports_coordinates[port][0], dict_bad_ports_coordinates[port][1]]), unit='mi')
                if distance < min_distance:
                    min_distance = distance
                    shortest_point_for_bad_ports[week][port] = coord
        
        else:
            continue
    

100%|██████████| 14/14 [00:34<00:00,  2.48s/it]


### Корректировка ошибки алгоритма (удаление лишних новых координат портов)

In [21]:
target_port = 'Индига'
target_coordinate = (27, 48)
ports_not_for_new_coordinates  = {}

for week in tqdm(list(week_data_dict.keys())):
    ports_not_for_new_coordinates[week] = []
    Graph = get_ice_graph(speed_matrix=np.array(week_data_dict[week]), max_personal_speed=21, ship_name='Ямал', ship_type='Arc 9')
    
    for port in dict_bad_ports_coordinates:
        min_distance = float('inf')
        path = get_path(graph=Graph, index_start=target_coordinate, index_end=dict_bad_ports_coordinates[port])
        if path != 'Нужен ледокол':
            ports_not_for_new_coordinates[week].append(port)

100%|██████████| 14/14 [00:32<00:00,  2.31s/it]


In [22]:
correct_shortest_point_for_bad_ports = {}
for week in shortest_point_for_bad_ports:
    correct_shortest_point_for_bad_ports[week] = {}
    for port in shortest_point_for_bad_ports[week]:
        if port not in ports_not_for_new_coordinates[week]:
            correct_shortest_point_for_bad_ports[week][port] = shortest_point_for_bad_ports[week][port]

In [23]:
def replace_bad_port_coordinates(row):
    for port in correct_shortest_point_for_bad_ports[week]:
        if row['name_start'] == port:
            row['graph_start_index'] = correct_shortest_point_for_bad_ports[week][port]
        if row['name_end'] == port:
            row['graph_end_index'] = correct_shortest_point_for_bad_ports[week][port]
    return row

In [24]:
dataset_with_normal_coordinates = {}
for week in list(correct_shortest_point_for_bad_ports.keys()):
    new_start_data =  start_data.apply(replace_bad_port_coordinates, axis=1)
    dataset_with_normal_coordinates[week] = new_start_data

### Построение новых маршрутов с верными координатами

In [25]:
ship_types = ['Нет', 'Arc 4', 'Arc 7', 'Arc 9']
ship_names = ['Имя_судна_1', 'Имя_судна_2', 'Имя_судня_3', 'Ямал']
ship_speeds = [14, 19, 19, 21, 18.5]

result_model_output = {}
for week in tqdm(week_data_dict):
    result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        result_model_output[week][sheep_types_name] = get_ice_graph(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])

 29%|██▊       | 4/14 [00:14<00:35,  3.59s/it]

100%|██████████| 14/14 [00:50<00:00,  3.64s/it]


In [27]:
len(list(result_model_output['03-Mar-2020']['Arc 9_Ямал'].nodes()))

8904

### Построение ледового графа

In [28]:
week_ice_graph = {}
for week in tqdm(result_model_output):
    week_ice_graph[week] = {}
    for graph in result_model_output[week]:
        main_graph = result_model_output[week]['Arc 9_Ямал']
        
        main_graph_copy = main_graph.copy()
        for node in list(main_graph_copy.nodes()):
            if node not in result_model_output[week][graph].nodes():
                continue
            elif main_graph_copy.degree(node) > result_model_output[week][graph].degree(node):
                continue
            else:
                main_graph_copy.remove_node(node)
        # main_graph_copy_save_format = nx.node_link_data(main_graph_copy)
        # week_ice_graph[week][graph] = main_graph_copy_save_format
        week_ice_graph[week][graph] = main_graph_copy

100%|██████████| 14/14 [00:08<00:00,  1.69it/s]


In [30]:
# with open('/home/kmulygin/khakaton/new_ice_graphs.pkl', 'wb') as f:
# 	pickle.dump(week_ice_graph, f)

### Построение граничных точек для всех типов графов и судов

In [24]:
week_ice_graph_boundary_points = {}
for week in tqdm(result_model_output):
    week_ice_graph_boundary_points[week] = {}
    for graph in result_model_output[week]:
        main_graph = result_model_output[week]['Arc 9_Ямал']
        if graph != 'Arc 9_Ямал':
            main_graph_copy = main_graph.copy()
            for node in list(main_graph_copy.nodes()):
                if node not in result_model_output[week][graph].nodes():
                    main_graph_copy.remove_node(node)
                elif main_graph_copy.degree(node) > result_model_output[week][graph].degree(node):
                    continue
                else:
                    main_graph_copy.remove_node(node)
            week_ice_graph_boundary_points[week][graph] = list(main_graph_copy.nodes())

100%|██████████| 14/14 [00:06<00:00,  2.16it/s]


In [28]:
# with open('/home/kmulygin/khakaton/ice_graph_boundary_points.pkl', 'wb') as f:
# 	pickle.dump(week_ice_graph_boundary_points, f)